In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

# Paths
original_dataset = "E:/Downloads/sign_language/SignVision/data_processed/crops/train"
augmented_dataset = "dataset_augmented"

# Recreate output folder cleanly
if os.path.exists(augmented_dataset):
    shutil.rmtree(augmented_dataset)
os.makedirs(augmented_dataset, exist_ok=True)

# Define augmentation generator
datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    fill_mode='nearest'
)

# Number of augmented images to create per input image
AUGS_PER_IMAGE = 5

for class_name in sorted(os.listdir(original_dataset)):
    class_dir = os.path.join(original_dataset, class_name)
    if not os.path.isdir(class_dir):
        continue
    
    save_dir = os.path.join(augmented_dataset, class_name)
    os.makedirs(save_dir, exist_ok=True)
    
    print(f"🔄 Augmenting class: {class_name}")
    
    for filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, filename)
        try:
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # Generate augmented images
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=save_dir, save_prefix="aug", save_format="jpg"):
                i += 1
                if i >= AUGS_PER_IMAGE:
                    break  # Stop after N augmentations

            # Also copy the original image
            shutil.copy(img_path, os.path.join(save_dir, filename))
        except Exception as e:
            print(f"⚠️ Skipping {filename}: {e}")

print("\n✅ Augmentation completed successfully!")


🔄 Augmenting class: 0
🔄 Augmenting class: 1
🔄 Augmenting class: 2
🔄 Augmenting class: 3
🔄 Augmenting class: 4

✅ Augmentation completed successfully!


In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import os

# Paths
train_dir = "dataset_augmented"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
NUM_CLASSES = 5  # 0–4

# Data generator
train_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Model: MobileNetV2 base
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers initially

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

# Unfreeze last layers and fine-tune
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

fine_tune_epochs = 10
history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=fine_tune_epochs
)

# Save model
os.makedirs("models", exist_ok=True)
model.save("models/signvision_cnn_augmented.h5")
print("\n✅ Model training completed and saved to 'models/signvision_cnn_augmented.h5'")


Found 493 images belonging to 5 classes.
Found 122 images belonging to 5 classes.


c:\Users\PC\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 677ms/step - accuracy: 0.1639 - loss: 1.9743 - val_accuracy: 0.4508 - val_loss: 1.3169
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 578ms/step - accuracy: 0.4407 - loss: 1.3768 - val_accuracy: 0.7541 - val_loss: 0.8866
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 600ms/step - accuracy: 0.6787 - loss: 0.9483 - val_accuracy: 0.9098 - val_loss: 0.6054
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 579ms/step - accuracy: 0.8143 - loss: 0.7399 - val_accuracy: 0.9344 - val_loss: 0.4245
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 596ms/step - accuracy: 0.8609 - loss: 0.5766 - val_accuracy: 0.9508 - val_loss: 0.3171
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 620ms/step - accuracy: 0.9234 - loss: 0.4202 - val_accuracy: 0.9590 - val_loss: 0.2405
Epoch 7/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 572ms/step - accuracy: 0.9121 - loss: 0.3961 - val_accuracy: 0.9590 - val_loss: 0.1958
Epoch 8/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 561ms/step - accuracy: 0.9348 - loss: 0.3131 - val_accuracy


✅ Model training completed and saved to 'models/signvision_cnn_augmented.h5'
